In [12]:
import tweepy
import json
import pandas as pd

pd.set_option('display.max_colwidth', -1)
# == OAuth Authentication ==
#
# This mode of authentication is the new preferred way
# of authenticating with Twitter.

secret = json.load(open('secret.json'))

# The consumer keys can be found on your application's Details
# page located at https://dev.twitter.com/apps (under "OAuth settings")
consumer_key=secret["consumer_key"]
consumer_secret=secret["consumer_secret"]

# The access tokens can be found on your applications's Details
# page located at https://dev.twitter.com/apps (located
# under "Your access token")
access_token=secret["access_token"]
access_token_secret=secret["access_token_secret"]

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())

# If the authentication was successful, you should
# see the name of the account print out
print(api.me())


# status = api.get_status(id, tweet_mode="extended")
# status

# a=api.search(q="bolivia",count=100,tweet_mode="extended",result_type="recent")


# b=api.

# If the application settings are set for "Read and Write" then
# this line should tweet out the message to your account's
# timeline. The "Read and Write" setting is on https://dev.twitter.com/apps
#api.update_status(status='Updating using OAuth authentication via Tweepy!')

{'id': 753865475533275137, 'id_str': '753865475533275137', 'name': 'Jorge Antequera', 'screen_name': 'JorgeAntequera_', 'location': '', 'profile_location': None, 'description': '', 'url': None, 'entities': {'description': {'urls': []}}, 'protected': False, 'followers_count': 13, 'friends_count': 178, 'listed_count': 0, 'created_at': 'Fri Jul 15 08:15:10 +0000 2016', 'favourites_count': 9, 'utc_offset': None, 'time_zone': None, 'geo_enabled': False, 'verified': False, 'statuses_count': 4, 'lang': None, 'status': {'created_at': 'Tue Nov 12 23:21:48 +0000 2019', 'id': 1194394895055773696, 'id_str': '1194394895055773696', 'text': 'RT @Almagro_OEA2015: ¿Golpe de Estado en #Bolivia?\n\nSí. En el momento que pretendieron robarse las elecciones del 20 de octubre y perpetuar…', 'truncated': False, 'entities': {'hashtags': [{'text': 'Bolivia', 'indices': [41, 49]}], 'symbols': [], 'user_mentions': [{'screen_name': 'Almagro_OEA2015', 'name': 'Luis Almagro', 'id': 2802545808, 'id_str': '2802545808

In [215]:
def transform(tweet_query_json):
    l=[]
    for tweet in tweet_query_json["statuses"]:
        try:
            l.append((tweet["retweeted_status"]["created_at"],\
                      tweet["retweeted_status"]["full_text"],tweet["retweeted_status"]["id"], \
                      tweet["id"],
                      tweet["retweeted_status"]["user"]["name"],tweet["retweeted_status"]["retweet_count"],\
                      tweet["retweeted_status"]["favorite_count"],tweet["retweeted_status"]["user"]["followers_count"],\
                      tweet["retweeted_status"]["user"]["friends_count"]))   
        except:
            l.append((tweet["created_at"],tweet["full_text"],tweet["id"],tweet["id"],tweet["user"]["name"],tweet["retweet_count"],\
                      tweet["favorite_count"],tweet["user"]["followers_count"],tweet["user"]["friends_count"]))


    df = pd.DataFrame(l,columns=["created_at","text","tweet_id","tweet_original_id","original_user","retweets","favs","followers","friends"])

    df = df.drop_duplicates(subset="tweet_id").reset_index(drop=True)
    
    min_id = df["tweet_original_id"].min()
    
    return [min_id,df]

def search_tweet(topic,max_id=None):
    json_tweet = api.search(q=topic,count=1000,lang='es',tweet_mode="extended",result_type="recent",max_id=max_id,wait_on_rate_limit=True)
    return json_tweet



# tup = transform(search_tweet("#NoHayGolpeEnBolivia"))
# tup[1]

t = transform(search_tweet("bolivia"))[1]

In [ ]:
# max_id=1191108460311728130
# api.search(q="bolivia",count=1000,tweet_mode="extended",result_type="recent",max_id=max_id,wait_on_rate_limit=True)

In [113]:
import time

while (tup[1].shape[0]<50000):
#     print(tup[0])
    df_temp = transform(search_tweet("#NoHayGolpeEnBolivia",max_id=tup[0]))[1] 
    tup[0] = df_temp["tweet_original_id"].min()
    tup[1] = tup[1].append(df_temp)\
    .drop_duplicates(subset="tweet_id")\
    .reset_index(drop=True)
    time.sleep(0.1)
    print(tup[0])

1198919811708964864
1198743385890336768
1198671995396861954
1198565263664390146
1198396258806841345
1198290871088627712
1198145139383902208
1198002875856818176
1197898162419093504
1197802214351396865
1197684982027825153
1197617006536728591
1197574836006129664
1197522603671506945
1197485084171014144
1197401338684092417
1197363913135992837
1197316090218655747
1197231134385541126
1197177658712449029
1197160242506846208
1197130685938634752
1197098404108783617
1196992508603117568
1196970793751732224
1196951420324454401
1196918011170631680
1196870426321915904
1196829002075451397
1196781595572748290
1196752493755207681
1196675971958657024
1196636787030024193
1196619059565076480
1196601792135647232
1196560147444850688
1196510593752195073
1196475046509854720
1196443594862800898
1196400878904852480
1196297726637608961
1196234984618708993
1196174120582598656
1196131033944666118
1196106547790786566
1196085473862725632
1196047671469002758
1196010719097114624
1196010719097114624
1196010719097114624


KeyboardInterrupt: 

In [129]:
# search_tweet("bolivia",max_id=1190432381230047237)
# transform(search_tweet("bolivia",max_id=tup[0]))[1]
# transform(search_tweet("bolivia",max_id=tup[0]))[1]["tweet_original_id"].min()
# df_temp = transform(search_tweet("bolivia",max_id=1191108460311728130))[1] 
# df_temp["tweet_original_id"].sort_values()
# df_temp2[["tweet_id","tweet_original_id"]].sort_values(by="tweet_id")
# df = tup[1]
# df.sort_values("favs",ascending = False)
df1=tup[1]
df

,created_at,text,tweet_id,tweet_original_id,original_user,retweets,favs,followers,friends,label,words
0,Tue Nov 26 13:11:00 +0000 2019,@noticiasfides @ArturoMurilloS Y las negociaciones con todos los sectores. #GolpeDeEstadoBolivia,1199314613931773952,1199314613931773952,ivan,0,0,0,16,0,"[@noticiasfides, @arturomurillos, y, las, negociaciones, con, todos, los, sectores]"
1,Tue Nov 26 13:10:33 +0000 2019,@Silvia6566 Buen día kumpa.. ❤💚💜✌💪 VAMOS CFK❤❤❤❤❤ #CuidemosNuestraDemocracia #GolpeDeEstadoBolivia #ChileSomosTodos,1199314498462597121,1199314498462597121,@Cheche62💚💚🍀🍀,0,0,2441,3099,0,"[@silvia6566, buen, día, kumpa, ❤💚💜✌💪, vamos, cfk❤❤❤❤❤, #cuidemosnuestrademocracia, #chilesomostodos]"
2,Sun Nov 24 17:38:03 +0000 2019,@evoespueblo @evoespueblo lo que no te perdonan los golpistas oligarquía rancia y Fascista es que eres un hombre demasiado Honesto y Digno que puedes verle los ojos directo a tu Pueblo iVolveras Hecho Millones! \n#JovenesDelMundoConVenezuela y Bolivia #GolpeDeEstadoBolivia https://t.co/Uf2cqiXFvy,1198657042183004161,1199314293768032257,🌹Vilma Picapiedras💞🌹,105,190,20387,6852,0,"[@evoespueblo, @evoespueblo, lo, que, no, te, perdonan, los, golpistas, oligarquía, rancia, y, fascista, es, que, eres, un, hombre, demasiado, honesto, y, digno, que, puedes, verle, los, ojos, directo, a, tu, pueblo, ivolveras, hecho, millones, #jovenesdelmundoconvenezuela, y, bolivia]"
3,Tue Nov 26 13:08:32 +0000 2019,#Chile\n\nLa @FundVictorJara emite #DeclaracionPublica respecto al uso indebido de la obra: El Derecho de Vivir en Paz de #VictorJara \n\n#GolpeDeEstadoBolivia \n#BoliviaSeRespeta\n#ElDerechoDeVivirEnPaz https://t.co/fgKDl7s0hh,1199313992746975234,1199313992746975234,FSLN Cls Nicaragua,0,0,3263,346,0,"[#chile, la, @fundvictorjara, emite, #declaracionpublica, respecto, al, uso, indebido, de, la, obra, el, derecho, de, vivir, en, paz, de, #victorjara, #boliviaserespeta, #elderechodevivirenpaz]"
4,Tue Nov 26 13:01:51 +0000 2019,"Asesinos de maestros era lo que quería el imperialismo para #Cuba, son los del #PlanCondor, los que masacran al pueblo en #ColombiaConP, #Chile, los del #GolpeDeEstadoBolivia etc. #TenemosMemoria #FidelPorSiempre https://t.co/AhNv9JNsF2",1199312311330254849,1199313837146685445,Viceversa,1,1,1573,2104,0,"[asesinos, de, maestros, era, lo, que, quería, el, imperialismo, para, #cuba, son, los, del, #plancondor, los, que, masacran, al, pueblo, en, #colombiaconp, #chile, los, del, etc, #tenemosmemoria, #fidelporsiempre]"
5,Mon Nov 25 20:01:44 +0000 2019,Tenemos la obligación de defender nuestro recurso Naturales \n#GolpeDeEstadoBolivia https://t.co/OLZfwbfXQv,1199055587562151937,1199313222827937792,HUGO,69,64,3110,4452,0,"[tenemos, la, obligación, de, defender, nuestro, recurso, naturales]"
6,Tue Nov 26 13:05:01 +0000 2019,"Estoy alucinando con la ""eficacia"" de la fiscalía, ¿tienen pruebas?. Se me olvida queremos un proceso de diálogo y paz pero por detrás hacemos casería da igual, la cuestión es venganza #GolpeDeEstadoBolivia https://t.co/JBmNZQhOtK",1199313108214468608,1199313108214468608,Giovana Ferrufino,0,0,86,373,0,"[estoy, alucinando, con, la, eficacia, de, la, fiscalía, tienen, pruebas, se, me, olvida, queremos, un, proceso, de, diálogo, y, paz, pero, por, detrás, hacemos, casería, da, igual, la, cuestión, es, venganza]"
7,Tue Nov 26 09:58:51 +0000 2019,En #Chile se VIOLAN los #DerechosHumanos con el #TerrorismoDeEstado y\n#ChileEnDictadura de #Piñera\n#Tarapacá #Iquique #AltoHospicio #Suecia #Francia #Canada #Noruega #BoliviaEnDictadura #MasacreEnBolivia #GolpeDeEstadoBolivia https://t.co/nTEI2knYlW,1199266255846486016,1199313055491985410,Crow,3,3,97,867,0,"[en, #chile, se, violan, los, #derechoshumanos, con, el, #terrorismodeestado, y, #chileendictadura, de, #piñera, #tarapacá, #iquique, #altohospicio, #suecia, #francia, #canada, #noruega, #boliviaendictadura, #masacreenbolivia]"
8,Wed Nov 13 00:44:58 +0000 2019,"Impugnación de elecciones legítimas. Injerencia de la OEA. Amotinamiento policial y

In [29]:
j =api.search(q='#NoHayGolpeEnBolivia',count=1000,lang='es',tweet_mode="extended",result_type="recent",wait_on_rate_limit=True)

In [132]:
# list(map(api.destroy_favorite,list(df["tweet_id"])))
del df["words"]

In [168]:
import re

r= re.compile("https.+\s?")

df_clean = (df["text"].str.replace('\n'," ")
            .str.lower()
#             .str.replace("#nohaygolpeenbolivia","")
            .str.replace("\n"," ")
            .str.replace(r,"")
            .str.replace("[\.\,\?\!\:\"\”\“\¿\-\🇧🇴\|]","")
            .str.replace("\s{2,}"," ")
            .str.strip()            
            .str.strip() 
            .str.split(" "))
df["words"] = df_clean
df['label'] = 0
# df.to_csv("Twitter/golpe.csv",sep='\t')
df_clean_1 = (df1["text"].str.replace('\n'," ")
            .str.lower()
#             .str.replace("#nohaygolpeenbolivia","")
            .str.replace("\n"," ")
            .str.replace("[\.\,\?\!\:\"\”\“\¿\-\🇧🇴\|]","")
            .str.replace("\s{2,}"," ")
            .str.strip()            
            .str.strip() 
            .str.split(" "))
df1["words"] = df_clean_1
df1['label'] = 1

df_final = df.append(df1).reset_index()
df_final[["text","words","label"]]
df_final["text"].str.contains(r)

0       False
1       False
2       True 
3       True 
4       True 
5       True 
6       True 
7       True 
8       True 
9       True 
10      False
11      False
12      True 
13      True 
14      True 
15      True 
16      True 
17      True 
18      True 
19      True 
20      True 
21      True 
22      True 
23      True 
24      True 
25      True 
26      False
27      False
28      True 
29      True 
        ...  
3080    True 
3081    True 
3082    True 
3083    True 
3084    True 
3085    True 
3086    False
3087    False
3088    False
3089    True 
3090    False
3091    False
3092    False
3093    False
3094    False
3095    True 
3096    False
3097    True 
3098    False
3099    False
3100    False
3101    False
3102    False
3103    False
3104    False
3105    False
3106    True 
3107    False
3108    False
3109    False
Name: text, Length: 3110, dtype: bool

In [219]:
# tup[1].sort_values(by="followers",ascending=False
# df_temp2

# df["split"]=(df["text"].str.lower()
#              .str.replace("\n"," ")
#              .str.replace("[\.\,\?\!\:\"\”\“\¿\-\🇧🇴\|]","")
#              .str.replace("\s{2,}"," ")
#              .str.strip()
#              .str.split(" "))

# df.sort_values("favs",ascending=False)
# df_final[["text","words","label"]].to_parquet("Twitter/data.parquet")
df_clean_t = (t["text"].str.replace('\n'," ")
            .str.lower()
#             .str.replace("#nohaygolpeenbolivia","")
            .str.replace("\n"," ")
            .str.replace("[\.\,\?\!\:\"\”\“\¿\-\🇧🇴\|]","")
            .str.replace("\s{2,}"," ")
            .str.strip()            
            .str.strip() 
            .str.split(" "))
t["words"] = df_clean_t
t['label'] = 2
t.to_parquet("Twitter/test.parquet")

In [220]:
# df_test = df_final[["text","words","label"]].iloc[0:2]
t

,created_at,text,tweet_id,tweet_original_id,original_user,retweets,favs,followers,friends,words,label
0,Tue Nov 26 15:31:07 +0000 2019,ÚLTIMA HORA | Canciller de Bolivia revela que mañana recibirán a la embajadora designada por el Pdte. Guaidó (VIDEO) https://t.co/JqGwx8ryjs,1199349873562308608,1199399932794458118,AlbertoNews,87,149,477031,21042,"[última, hora, canciller, de, bolivia, revela, que, mañana, recibirán, a, la, embajadora, designada, por, el, pdte, guaidó, (video), https//tco/jqgwx8ryjs]",2
1,Mon Nov 25 23:52:12 +0000 2019,"Lo de los trolls bancando al golpe de Estado en Bolivia es un escándalo, @TwitterLatAm. Tienen que hacer algo con esas 60 mil cuentas creadas con la única finalidad de apoyar a un gobierno de facto como el de Jeanine Añez.",1199113587949809669,1199399929959067649,Juan Manuel Karg,1136,1984,43579,3655,"[lo, de, los, trolls, bancando, al, golpe, de, estado, en, bolivia, es, un, escándalo, @twitterlatam, tienen, que, hacer, algo, con, esas, 60, mil, cuentas, creadas, con, la, única, finalidad, de, apoyar, a, un, gobierno, de, facto, como, el, de, jeanine, añez]",2
2,Tue Nov 26 18:50:01 +0000 2019,"@Marvazquez92 @Sebasti67180134 @renandelcent @ignaciojose4221 @darwindice @jmkarg Así q métete tu pensamiento de socialismo bien en el orto, y aporta a tu País que de seguro eres un parásito que ni trabajo tienes y si quieras opinar de Bolivia por lo menos vení a conocer una de sus ciudades, seguro no conoces ni la frontera pero el boludo opinando!!",1199399928348454913,1199399928348454913,Carlos Delgado Centellas,0,0,18,22,"[@marvazquez92, @sebasti67180134, @renandelcent, @ignaciojose4221, @darwindice, @jmkarg, así, q, métete, tu, pensamiento, de, socialismo, bien, en, el, orto, y, aporta, a, tu, país, que, de, seguro, eres, un, parásito, que, ni, trabajo, tienes, y, si, quieras, opinar, de, bolivia, por, lo, menos, vení, a, conocer, una, de, sus, ciudades, seguro, no, conoces, ni, la, frontera, pero, el, boludo, opinando]",2
3,Tue Nov 26 16:20:47 +0000 2019,"Michelle Bachelet mantiene un silencio complaciente ante la violación flagrante de los Derechos Humanos de los pueblos de Chile, Bolivia y Colombia. ¿Dónde está su pronunciamiento frente a la cantidad de muertos en esos países?",1199362374488207365,1199399924707844107,Carmen Zambrano,49,19,10317,9603,"[michelle, bachelet, mantiene, un, silencio, complaciente, ante, la, violación, flagrante, de, los, derechos, humanos, de, los, pueblos, de, chile, bolivia, y, colombia, dónde, está, su, pronunciamiento, frente, a, la, cantidad, de, muertos, en, esos, países]",2
4,Tue Nov 26 17:07:55 +0000 2019,El embajador aseguró que la policía boliviana arresta a dirigentes políticos inocentes https://t.co/OXh1gbHhZN,1199374234687963136,1199399923143315463,Últimas Noticias,8,5,3974667,101662,"[el, embajador, aseguró, que, la, policía, boliviana, arresta, a, dirigentes, políticos, inocentes, https//tco/oxh1gbhhzn]",2
5,Tue Nov 26 17:20:00 +0000 2019,#NotiMippCI 📰🗞| Gobierno de facto de Bolivia inició persecución política contra figuras cercanas a Evo Morales. Lea más ⏩ https://t.co/SGvActbnKC #GMVVHito2Millones900Mil https://t.co/G3mcUQIlJ7,1199377274392530944,1199399923105427456,MIPPCI,50,5,115722,711,"[#notimippci, 📰🗞, gobierno, de, facto, de, bolivia, inició, persecución, política, contra, figuras, cercanas, a, evo, morales, lea, más, ⏩, https//tco/sgvactbnkc, #gmvvhito2millones900mil, https//tco/g3mcuqilj7]",2
6,Mon Nov 25 21:19:14 +0000 2019,"#Bolivia : En menos de 2 semanas, la ministra de comunicación acusa a periodistas de sedición y les advierte atenerse a la ley; y hoy, sacan a @teleSURtv de la transmisión. Es tanta democracia para no creer.",1199075092623437826,1199399919976681472,Adriana Salvatierra,2928,4548,58406,392,"[#bolivia, en, menos, de, 2, semanas, la, ministra, de, comunicación, acusa, a, periodistas, de, sedición, y, les, advierte, atenerse, a, la, ley;, y, hoy, sacan, a, @telesurtv, de, la, transmisión, es, tanta, democracia, para, no, creer]"

In [214]:
df_test.loc[9] = ["Quienes apoyan el golpe en Bolivia fueron a gritarle a las víctimas de la represión antes de dar testimonio a la CIDH \
Lo mismo le hicieron a las Madres de Plaza de Mayo cuando la CIDH visitó Argentina en el medio de la dictadura \
Inhumano, grotesco y miserable",[],0]
df_test.to_parquet("Twitter/test.parquet")
df_test

,text,words,label
0,Maldito Evo,"[@noticiasfides, @arturomurillos, y, las, negociaciones, con, todos, los, sectores, #golpedeestadobolivia]",0
1,Amado evo,"[@silvia6566, buen, día, kumpa, ❤💚💜✌💪, vamos, cfk❤❤❤❤❤, #cuidemosnuestrademocracia, #golpedeestadobolivia, #chilesomostodos]",0
2,Evo asesino,[],0
3,Evo evo cabron,[],0
4,Evo asesino,[],0
6,A mi me parece que evo morales se voplvio un dicatador de los peores. #nohaygolpeenbolivia,[],0
7,"Presidente Ev, toda mi solidaridad en estos duros moments. #evonoestassolo",[],0
8,@evoespueblo @evoespueblo lo que no te perdonan los golpistas oligarquía rancia y Fascista es que eres un hombre demasiado Honesto y Digno que puedes verle los ojos directo a tu Pueblo iVolveras Hecho Millones! \n#JovenesDelMundoConVenezuela y Bolivia #GolpeDeEstadoBolivia,[],0
9,"Quienes apoyan el golpe en Bolivia fueron a gritarle a las víctimas de la represión antes de dar testimonio a la CIDH Lo mismo le hicieron a las Madres de Plaza de Mayo cuando la CIDH visitó Argentina en el medio de la dictadura Inhumano, grotesco y miserable",[],0


In [210]:
df_test

,text,words,label
0,Maldito Evo,"[@noticiasfides, @arturomurillos, y, las, negociaciones, con, todos, los, sectores, #golpedeestadobolivia]",0
1,Amado evo,"[@silvia6566, buen, día, kumpa, ❤💚💜✌💪, vamos, cfk❤❤❤❤❤, #cuidemosnuestrademocracia, #golpedeestadobolivia, #chilesomostodos]",0
2,Evo asesino,[],0
3,Evo evo cabron,[],0
4,Evo asesino,[],0
6,A mi me parece que evo morales se voplvio un dicatador de los peores. #nohaygolpeenbolivia,[],0
7,"Presidente Ev, toda mi solidaridad en estos duros moments. #evonoestassolo",[],0
